In [134]:

from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(
    os.environ.get('WORK_DIR')) #type: ignore
sys.path.append(
    os.environ.get('DBASE_DIR')) #type: ignore
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from EventDriven.data import HistoricTradeDataHandler
from EventDriven.event import *
from queue import Queue
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import BBandsTrend2
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from EventDriven.backtest import OptionSignalBacktest
from copy import deepcopy
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
thetadata_start = '2021-01-01'

In [136]:
import json
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
key = 9
with open(f'../input/profitable_weights_{key}.json', 'r') as f:
    weights = json.load(f)
ttrades__ = pd.read_csv(f'../input/profitable_trades_{key}.csv').iloc[:, 1:]


In [137]:
trades_ = ttrades__.copy()
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,21.0,506.0,508.0,183.269210,171.995209,-236.754013,-0.061516,2022-01-04,2022-01-06,2 days,AAPL
1,6.0,506.0,509.0,566.204785,536.949803,-175.529896,-0.051669,2022-01-04,2022-01-07,3 days,COST
2,5.0,506.0,510.0,410.070255,387.399994,-113.351304,-0.055284,2022-01-04,2022-01-10,6 days,HD
3,27.0,506.0,517.0,95.934598,88.466712,-201.632946,-0.077844,2022-01-04,2022-01-20,16 days,MU
4,21.0,509.0,519.0,173.495114,160.020004,-282.977312,-0.077669,2022-01-07,2022-01-24,17 days,AAPL
5,6.0,510.0,514.0,531.142493,506.350825,-148.750006,-0.046676,2022-01-10,2022-01-14,4 days,COST
6,28.0,512.0,515.0,49.342093,46.451465,-80.937601,-0.058583,2022-01-12,2022-01-18,6 days,BAC
7,17.0,515.0,517.0,137.228625,130.272318,-118.257227,-0.050691,2022-01-18,2022-01-20,2 days,TSM
8,6.0,515.0,517.0,496.190601,484.891153,-67.796691,-0.022772,2022-01-18,2022-01-20,2 days,COST
9,5.0,516.0,519.0,368.465123,343.779999,-123.425619,-0.066994,2022-01-19,2022-01-24,5 days,HD


In [138]:
symbol_list = trades_.Ticker.unique()
untraded_symbols = [s for s in weights.keys() if s not in trades_.Ticker.unique()]
for s in untraded_symbols:
    weights.pop(s)
pd.Series(weights).sort_values(ascending=False)

TSLA    0.128666
MSFT    0.096034
AAPL    0.079932
COST    0.072019
MU      0.052662
TSM     0.047082
HD      0.044737
BA      0.044120
SBUX    0.039109
CAT     0.039076
BAC     0.028146
GLD     0.018185
UNG     0.007615
dtype: float64

In [139]:

from EventDriven.riskmanager import RiskManager, close_cache, spot_cache, chain_cache, oi_cache, LOOKBACKS, order_cache
from pandas.tseries.offsets import BDay

rm = RiskManager(None, None, 1000000)
rm.OrderPicker.liquidity_threshold = 2
rm.OrderPicker.lookback = 10
rm.OrderPicker.data_availability_threshold = 0.15
date, tick = '2023-07-05', 'AVGO'
date, tick = '2024-08-13', 'TSM'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'
order_settings = {'type': 'spread',
 'specifics': [{'direction': 'long',
   'rel_strike': .50,
   'dte': 210,
   'moneyness_width': 0.5},
  #  {'direction': 'short',
  # 'rel_strike': .60,
  # 'dte': 270,
  # 'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}



order =  rm.OrderPicker.get_order(tick, date, right, 2, order_settings)
order

{'result': 'SUCCESSFUL',
 'data': {'long': ['TSM20250321C280'],
  'trade_id': '&L:TSM20250321C280',
  'close': 1.515}}

In [140]:
max_cash = {}
cash = 20_000
for s, w in weights.items():
    if w * cash > 200:
        max_cash[s] = 2
    elif w * cash > 100:
        max_cash[s] = 1
    else:
        max_cash[s] = 0.5
max_cash

{'TSLA': 2,
 'MSFT': 2,
 'AAPL': 2,
 'MU': 2,
 'COST': 2,
 'TSM': 2,
 'BA': 2,
 'HD': 2,
 'SBUX': 2,
 'CAT': 2,
 'BAC': 2,
 'UNG': 1,
 'GLD': 2}

In [141]:
#Backtest class 
## Find a way to not always reinitialize the backtest class, when want to redo
evb_backtest = OptionSignalBacktest(trades_, initial_capital=cash)

Updating db rates data


In [142]:
evb_backtest.portfolio.initial_capital

20000.0

In [145]:
w_map = {x: w  * 0.75 for x, w in weights.items()} ## 75% of the weights for each stock
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.risk_manager.OrderPicker.liquidity_threshold = 100
evb_backtest.portfolio.risk_manager.OrderPicker.lookback = 10
evb_backtest.portfolio.risk_manager.OrderPicker.data_availability_threshold = 0.5
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .850,
   'dte': 270,
   'moneyness_width': 0.35},
   {'direction': 'short',
  'rel_strike': .60,
  'dte': 270,
  'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}

# evb_backtest.portfolio.order_settings = {'type': 'naked',
#  'specifics': [{'direction': 'long',
#    'rel_strike': .750,
#    'dte': 210,
#    'moneyness_width': 0.5},

# ],
#  'name': 'vertical_spread'}


evb_backtest.portfolio.max_contract_price = max_cash
evb_backtest.executor.commission_rate = 0.65/100
evb_backtest.executor.commission_rate
evb_backtest.executor.max_slippage_pct = 0.075
evb_backtest.portfolio.roll_map = 250
evb_backtest.portfolio.allocated_cash_map
evb_backtest.portfolio.max_contract_price
evb_backtest.portfolio.roll_map

{'AAPL': 250,
 'COST': 250,
 'HD': 250,
 'MU': 250,
 'BAC': 250,
 'TSM': 250,
 'MSFT': 250,
 'GLD': 250,
 'TSLA': 250,
 'UNG': 250,
 'SBUX': 250,
 'CAT': 250,
 'BA': 250}

In [161]:
signals = evb_backtest.bars.signal_df
series = signals.set_index('Date')['MSFT']
series[series != 0]

Date
2022-01-21    1
2022-01-24   -1
2022-01-25   -1
2022-01-27    1
2022-02-14   -1
2022-02-16   -1
2022-02-17   -1
2022-03-23   -1
Name: MSFT, dtype: int64

In [147]:
signals_df = deepcopy(signals).set_index('Date')
signals_df[signals_df!=-1].sum().sum()

31.0

In [148]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

        Date  AAPL  COST  HD  MU  BAC  TSM  MSFT  GLD  TSLA  UNG  SBUX  CAT  \
0 2022-01-04     1     1   1   1    0    0     0    0     0    0     0    0   

   BA  
0   0  
Processing event: MARKET
Processing event: SIGNAL
Processing event: SIGNAL
Not generating order because:TOO_ILLIQUID SignalEvent type:LONG, symbol=COST, date:2022-01-04 00:00:00, trade_id:COST20220104C, Order Settings=None
Processing event: SIGNAL
Not generating order because:TOO_ILLIQUID SignalEvent type:LONG, symbol=HD, date:2022-01-04 00:00:00, trade_id:HD20220104C, Order Settings=None
Processing event: SIGNAL
Processing event: ORDER
Processing event: ORDER
Processing event: FILL
Processing event: FILL
Event queue is empty, processed 9 event(s)
Advancing to next date: 2022-01-05
        Date  AAPL  COST  HD  MU  BAC  TSM  MSFT  GLD  TSLA  UNG  SBUX  CAT  \
1 2022-01-05     0     0   0   0    0    0     0    0     0    0     0    0   

   BA  
1   0  
Processing event: MARKET
Event queue is empty, processed 1 ev

KeyError: 'pnl'

In [163]:
for positio, whole in evb_backtest.portfolio._OptionSignalPortfolio__trades.items():
    if positio == '&L:MSFT20220916C370&S:MSFT20220916C385':
        continue
    print(positio)
    print(whole['pnl'])
    print('')

&L:AAPL20220916C210&S:AAPL20220916C220
-106.18300649785866

&L:MU20220916C100&S:MU20220916C105
-62.092960877754365

&L:TSM20220916C140&S:TSM20220916C145
-49.96978426202608

&L:BAC20221021C42&S:BAC20221021C50
nan

&L:TSM20220916C145&S:TSM20220916C155
-31.7738395414745

&L:MSFT20220916C380&S:MSFT20220916C400
-129.75637149420805

&L:AAPL20221021C200&S:AAPL20221021C210
-53.1233905066091

&L:MSFT20220916C365&S:MSFT20220916C380
-105.32385094867692

&L:COST20230120C700&S:COST20230120C720
-113.52465910391857

&L:MU20221021C100&S:MU20221021C110
-35.814418475026514

&L:AAPL20221021C190&S:AAPL20221021C195
-118.84684540270086

&L:GLD20221230C195&S:GLD20221230C205
-13.170875118207334

&L:AAPL20221118C185&S:AAPL20221118C195
-116.35869072994024

&L:TSLA20221216C1650&S:TSLA20221216C1700
-149.3253338395146

&L:COST20230120C680&S:COST20230120C700
-96.36519599386861

&L:TSLA20221216C1600&S:TSLA20221216C1700
-155.02359165198698

&L:TSLA20221216C1700&S:TSLA20221216C1800
-75.81758131082745

&L:AAPL20230616C

In [154]:
evb_backtest.portfolio._OptionSignalPortfolio__trades['&L:MSFT20220916C370&S:MSFT20220916C385']

{'entry_price': 201.0944980183905,
 'entry_date': Timestamp('2022-01-21 00:00:00'),
 'quantity': 5,
 'symbol': 'MSFT',
 'entry_commission': 6.5,
 'entry_market_value': 998.9724900919526,
 'entry_slippage': 23.97249009195246,
 'total_entry_cost': 10.054724900919526,
 'auxilary_entry_cost': 30.47249009195246,
 'signal_id': 'MSFT20220121C',
 'trade_id': '&L:MSFT20220916C370&S:MSFT20220916C385'}

In [162]:
evb_backtest.portfolio.plot_portfolio()

KeyError: 'pnl'

In [80]:
evb_backtest.portfolio.aggregate()

AttributeError: 'DataFrame' object has no attribute 'ReturnPct'

In [48]:
evb_backtest.portfolio.trades

,Ticker,PnL,ReturnPct,EntryPrice,EntryCommission,EntrySlippage,EntryMarketValue,TotalEntryCost,AuxilaryEntryCost,ExitPrice,ExitCommission,ExitSlippage,ExitMarketValue,TotalExitCost,AuxilaryExitCost,Quantity,EntryTime,ExitTime,Duration,Positions,SignalID
0,AAPL,-810.127534,-4.178392,193.884981,6.5,25.424905,962.924905,969.424905,31.924905,159.297370,6.5,-47.013148,802.986852,796.486852,53.513148,5,2022-01-04,2022-01-06,2,&L:AAPL20220916C210&S:AAPL20220916C220,AAPL20220104C
1,MU,-457.904886,-2.269172,201.793861,3.9,16.481583,601.481583,605.381583,20.381583,147.476697,3.9,-26.169908,446.330092,442.430092,30.069908,3,2022-01-04,2022-01-20,16,&L:MU20220916C100&S:MU20220916C105,MU20220104C
2,COST,-62.284835,-0.162419,383.482402,1.3,9.682402,382.182402,383.482402,10.982402,321.197567,1.3,-17.502433,322.497567,321.197567,18.802433,1,2022-04-05,2022-04-29,24,&L:COST20230120C680&S:COST20230120C700,COST20220405C
3,COST,-361.527791,-1.034702,349.402930,2.6,31.205861,696.205861,698.805861,33.805861,337.278069,2.6,-32.843861,677.156139,674.556139,35.443861,2,2022-01-10,2022-01-14,4,&L:COST20230120C560&S:COST20230120C570,COST20220110C
4,COST,-442.400087,-1.232661,358.898373,2.6,35.196746,715.196746,717.796746,37.796746,275.396659,2.6,-41.606682,553.393318,550.793318,44.206682,2,2022-01-18,2022-01-20,2,&L:COST20230120C540&S:COST20230120C550,COST20220118C
5,TSM,-426.870311,-2.204257,193.657194,3.9,29.571582,577.071582,580.971582,33.471582,154.101271,3.9,-21.296187,466.203813,462.303813,25.196187,3,2022-01-18,2022-01-20,2,&L:TSM20220916C140&S:TSM20220916C145,TSM20220118C
6,HD,-441.258227,-2.040509,216.249081,3.9,44.847244,644.847244,648.747244,48.747244,207.489017,3.9,-18.632950,626.367050,622.467050,22.532950,3,2022-01-19,2022-01-24,5,&L:HD20230120C420&S:HD20230120C430,HD20220119C
7,BAC,NaN,NaN,NaN,0.0,0.000000,0.000000,NaN,0.000000,NaN,0.0,-0.000000,0.000000,NaN,0.000000,0,2022-01-19,2022-01-24,5,&L:BAC20221021C42&S:BAC20221021C50,BAC20220119C
8,TSM,-265.206651,-1.258266,210.771599,2.6,18.943197,418.943197,421.543197,21.543197,156.336546,2.6,-24.726908,315.273092,312.673092,27.326908,2,2022-01-21,2022-01-27,6,&L:TSM20230120C150&S:TSM20220916C145,TSM20220121C
9,MSFT,-1098.846926,-5.117004,214.744218,7.8,80.665310,1280.665310,1288.465310,88.465310,189.618384,7.8,-54.489696,1145.510304,1137.710304,62.289696,6,2022-01-21,2022-01-24,3,&L:MSFT20230120C385&S:MSFT20220916C365,MSFT20220121C


In [49]:
evb_backtest.portfolio.get_weighted_holdings()

,AAPL,COST,HD,MU,BAC,TSM,MSFT,GLD,TSLA,UNG,SBUX,CAT,BA,cash,commission,total
datetime,,,,,,,,,,,,,,,,
2022-01-04,1198.980017,1080.288130,671.056371,789.926183,422.192655,706.235358,1440.510835,272.777887,1929.996497,114.221328,586.633211,586.139486,661.794728,9539.247313,0.000,20000.000000
2022-01-04,1167.055112,1042.498855,671.056371,769.544600,422.192655,706.235358,1440.510835,272.777887,1929.996497,114.221328,586.633211,586.139486,661.794728,9539.247313,0.130,19909.904237
2022-01-05,1079.555112,1257.498855,671.056371,687.044600,422.192655,706.235358,1440.510835,272.777887,1929.996497,114.221328,586.633211,586.139486,661.794728,9539.247313,0.130,19954.904237
2022-01-06,1026.041965,1037.498855,671.056371,732.044600,422.192655,706.235358,1440.510835,272.777887,1929.996497,114.221328,586.633211,586.139486,661.794728,9539.247313,0.195,19726.391089
2022-01-07,1026.041965,1011.499666,671.056371,717.044600,422.192655,706.235358,1440.510835,272.777887,1929.996497,114.221328,586.633211,586.139486,661.794728,9539.247313,0.221,19685.391901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,694.172412,763.408019,277.655708,431.852010,422.192655,478.697484,926.376614,201.008004,1559.621991,114.221328,586.633211,681.934346,838.505042,9539.247313,1.508,17515.526139
2022-12-27,694.172412,763.408019,277.655708,431.852010,422.192655,478.697484,926.376614,201.008004,1559.621991,114.221328,586.633211,681.934346,838.505042,9539.247313,1.508,17515.526139
2022-12-28,694.172412,763.408019,277.655708,431.852010,422.192655,478.697484,926.376614,201.008004,1559.621991,114.221328,586.633211,654.434346,823.505042,9539.247313,1.508,17473.026139


In [50]:

pd.set_option('display.max_rows', 10000)
evb_backtest.portfolio.get_all_positions()

long                short  \
datetime   symbol                                             
2022-01-04 AAPL     [AAPL20220916C210]   [AAPL20220916C220]   
           COST     [COST20230120C680]   [COST20230120C700]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-05 AAPL     [AAPL20220916C210]   [AAPL20220916C220]   
           COST     [COST20230120C680]   [COST20230120C700]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-06 COST     [COST20230120C680]   [COST20230120C700]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-07 MU         [MU20220916C100]     [MU20220916C105]   
2022-01-10 COST     [COST20230120C560]   [COST20230120C570]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-11 COST     [COST20230120C560]   [COST20230120C570]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-12 COST     [COST20230120C560]   [COST20230120C570]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-13 COST     [COST20230120C560]   [COST20230120C570]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-14 MU         [MU20220916C100]     [MU20220916C105]   
2022-01-17 MU         [MU20220916C100]     [MU20220916C105]   
2022-01-18 COST     [COST20230120C540]   [COST20230120C550]   
           MU         [MU20220916C100]     [MU20220916C105]   
           TSM       [TSM20220916C140]    [TSM20220916C145]   
2022-01-19 COST     [COST20230120C540]   [COST20230120C550]   
           HD         [HD20230120C420]     [HD20230120C430]   
           MU         [MU20220916C100]     [MU20220916C105]   
           BAC        [BAC20221021C42]     [BAC20221021C50]   
           TSM       [TSM20220916C140]    [TSM20220916C145]   
2022-01-20 HD         [HD20230120C420]     [HD20230120C430]   
           BAC        [BAC20221021C42]     [BAC20221021C50]   
2022-01-21 HD         [HD20230120C420]     [HD20230120C430]   
           BAC        [BAC20221021C42]     [BAC20221021C50]   
           TSM       [TSM20230120C150]    [TSM20220916C145]   
           MSFT     [MSFT20230120C385]   [MSFT20220916C365]   
2022-01-24 TSM       [TSM20230120C150]    [TSM20220916C145]   
2022-01-25 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           TSM       [TSM20230120C150]    [TSM20220916C145]   
2022-01-26 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           TSM       [TSM20230120C150]    [TSM20220916C145]   
2022-01-27 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-01-28 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-01-31 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-01 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           COST     [COST20230120C680]   [COST20230120C740]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-02 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           COST     [COST20230120C680]   [COST20230120C740]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-03 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           COST     [COST20230120C680]   [COST20230120C740]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-04 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           COST     [COST20230120C680]   [COST20230120C740]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-07 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
  

In [373]:


""""
Why do these two return weird stuff after run?
    current_weighted_holdings
    current_positions

I can't reconcile the cost with the data (NVM, haha)

"""
# evb_backtest.portfolio.all_positions

'"\nWhy do these two return weird stuff after run?\n    current_weighted_holdings\n    current_positions\n\nI can\'t reconcile the cost with the data (NVM, haha)\n\n'

In [269]:
order_cache['2024-01-03']

{'BA': {'result': 'TOO_ILLIQUID', 'data': None},
 'BAC': {'result': 'SUCCESSFUL',
  'data': {'long': ['BAC20240719C32'],
   'trade_id': '&L:BAC20240719C32',
   'close': 3.2}},
 'INTC': {'result': 'SUCCESSFUL',
  'data': {'long': ['INTC20240719C39'],
   'trade_id': '&L:INTC20240719C39',
   'close': 12.25}},
 'HD': {'result': 'TOO_ILLIQUID', 'data': None},
 'AMD': {'result': 'SUCCESSFUL',
  'data': {'long': ['AMD20240719C135'],
   'trade_id': '&L:AMD20240719C135',
   'close': 16.625}},
 'QCOM': {'result': 'SUCCESSFUL',
  'data': {'long': ['QCOM20240719C140'],
   'trade_id': '&L:QCOM20240719C140',
   'close': 45.9}},
 'MU': {'result': 'SUCCESSFUL',
  'data': {'long': ['MU20240719C85'],
   'trade_id': '&L:MU20240719C85',
   'close': 29.375}}}

## Extend for  get_port_stats
- Buy & Hold
- Dates
- Trades
- _strategy in Aggregate
- The function

In [214]:
evb_backtest.trades.to_csv(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/output/profitable_trades_options_{key}.csv')